In [1]:
import screed
import os
import pandas as pd

In [2]:
dammit_conversion_dir = "/kfish_annotations/namemaps/reformatted/"
basedir = "/kfish_clust/pep/"
renamed = "/kfish_clust/pep_renamed/"
assemblies = os.listdir(basedir)

In [3]:
print(assemblies)

['L_parva.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_diaphanus.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_notti.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_heteroclitusMDPL.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'A_xenica.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_notatus.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_catanatus.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'L_goodei.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_grandis.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_rathbuni.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_similis.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_chrysotus.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_sciadicus.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_parvapinis.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_heteroclitusMDPP.trinity_out.Trinity.fasta.transdecoder.pep.fasta', 'F_olivaceous.trinity_out.Trinity.fasta.trans

# test code

In [4]:
conversion_file = dammit_conversion_dir + "L_parva" + ".dammit.Trinity.conversions.csv"
conversions = pd.read_csv(conversion_file)
conversions = conversions.drop(columns=['Unnamed: 0'])
conversions = conversions.set_index('dammitID')

In [5]:
conversions.head()

,TrinityID
dammitID,
Transcript_0,TRINITY_DN8_c0_g1_i27
Transcript_1,TRINITY_DN8_c0_g1_i1
Transcript_2,TRINITY_DN8_c0_g1_i21
Transcript_3,TRINITY_DN8_c0_g1_i24
Transcript_4,TRINITY_DN8_c0_g1_i2


In [21]:
fasta_file = basedir + "F_diaphanus.trinity_out.Trinity.fasta.transdecoder.pep.fasta"
new_filename = renamed + "Fdiaphanus.pep.fasta"
genusspecies="Fdiaphanus"
total_records = 0
converted_records = 0
matchIDs = {}
for n,record in enumerate(screed.open(fasta_file)):
    if n<10:
        total_records +=1
        record_info = record.name.split("::")
        dammitID = record_info[1]
        type_info = record_info[5].split(" ")
        orf_type = type_info[3].split(":")[1]
        length = int(type_info[4].split(":")[1])
        TrinityID = conversions.loc[dammitID,'TrinityID']
        info = (record,dammitID,orf_type,length)
        if TrinityID in matchIDs:
            matchIDs[TrinityID].append(info)
        else:
            matchIDs[TrinityID] = [info]

['Gene.217', 'Transcript_100', 'g.217', 'm.217 Gene.217', 'Transcript_100', 'g.217  ORF type:3prime_partial len:290 (+) Transcript_100:62-928(+)']
['Gene.220536', 'Transcript_100000', 'g.220536', 'm.220536 Gene.220536', 'Transcript_100000', 'g.220536  ORF type:complete len:115 (+) Transcript_100000:1741-2085(+)']
['Gene.220539', 'Transcript_100001', 'g.220539', 'm.220539 Gene.220539', 'Transcript_100001', 'g.220539  ORF type:5prime_partial len:138 (+) Transcript_100001:3-416(+)']
['Gene.220540', 'Transcript_100001', 'g.220540', 'm.220540 Gene.220540', 'Transcript_100001', 'g.220540  ORF type:complete len:115 (+) Transcript_100001:1933-2277(+)']
['Gene.220542', 'Transcript_100002', 'g.220542', 'm.220542 Gene.220542', 'Transcript_100002', 'g.220542  ORF type:complete len:299 (+) Transcript_100002:74-970(+)']
['Gene.220544', 'Transcript_100003', 'g.220544', 'm.220544 Gene.220544', 'Transcript_100003', 'g.220544  ORF type:complete len:168 (+) Transcript_100003:439-942(+)']
['Gene.220547', 

In [7]:
with open(new_filename,'w') as fp:
    for TrinityID in matchIDs:
        complete = []
        partial = []
        if len(matchIDs[TrinityID])>1:
            converted_records +=1
            for i in matchIDs[TrinityID]:
                # check if complete
                if i[2] == 'complete':
                    complete.append(i)
                else:
                    partial.append(i)
            #check if complete is empty
            if len(complete) > 0:
                complete = sorted(complete, key=lambda x: x[3], reverse=True)
                chosen = complete[0]
            else:
                partial = sorted(partial, key=lambda x: x[3], reverse=True)
                chosen = partial[0]

            name = """{}-{}""".format(genusspecies,TrinityID)
            sequence = chosen[0]['sequence']
        else:
            name = """{}-{}""".format(genusspecies,TrinityID)
            sequence = matchIDs[TrinityID][0][0]['sequence']
        fp.write(">{name}\n{seq}\n".format(name=name, seq=sequence))
print("Written:",new_filename)

Written: /kfish_clust/pep_renamed/Lparva.pep.fasta


In [8]:
print(converted_records)

29736


In [9]:
print(total_records)

158788


# loop over all files

In [22]:
for fasta_filename in assemblies:
    if fasta_filename.endswith(".pep.fasta"):
        genus_species = fasta_filename.split(".")[0]
        print(genus_species)
        genusspecies = genus_species.split("_")[0] + genus_species.split("_")[1]
        conversion_file = dammit_conversion_dir + genus_species + ".dammit.Trinity.conversions.csv"
        conversions = pd.read_csv(conversion_file)
        conversions = conversions.drop(columns=['Unnamed: 0'])
        conversions = conversions.set_index('dammitID')
        fasta_file = basedir + fasta_filename
        new_filename = renamed + genusspecies + ".pep.fasta"
        total_records = 0
        converted_records = 0
        matchIDs = {}
        for n,record in enumerate(screed.open(fasta_file)):
            total_records +=1
            record_info = record.name.split("::")
            dammitID = record_info[1]
            type_info = record_info[5].split(" ")
            orf_type = type_info[3].split(":")[1]
            length = int(type_info[4].split(":")[1])
            TrinityID = conversions.loc[dammitID,'TrinityID']
            info = (record,dammitID,orf_type,length)
            if TrinityID in matchIDs:
                matchIDs[TrinityID].append(info)
            else:
                matchIDs[TrinityID] = [info]
        with open(new_filename,'w') as fp:
            for TrinityID in matchIDs:
                complete = []
                partial = []
                if len(matchIDs[TrinityID])>1:
                    converted_records +=1
                    for i in matchIDs[TrinityID]:
                        # check if complete
                        if i[2] == 'complete':
                            complete.append(i)
                        else:
                            partial.append(i)
                    #check if complete is empty
                    if len(complete) > 0:
                        complete = sorted(complete, key=lambda x: x[3], reverse=True)
                        chosen = complete[0]
                    else:
                        partial = sorted(partial, key=lambda x: x[3], reverse=True)
                        chosen = partial[0]
                    name = """{}-{}""".format(genusspecies,TrinityID)
                    sequence = chosen[0]['sequence']
                else:
                    name = """{}-{}""".format(genusspecies,TrinityID)
                    sequence = matchIDs[TrinityID][0][0]['sequence']
                fp.write(">{name}\n{seq}\n".format(name=name, seq=sequence))
        print("Total records:",total_records)
        print("Converted records:",converted_records)
        print("Written:",new_filename)

L_parva
Total records: 158788
Converted records: 29736
Written: /kfish_clust/pep_renamed/Lparva.pep.fasta
F_diaphanus
Total records: 124524
Converted records: 21965
Written: /kfish_clust/pep_renamed/Fdiaphanus.pep.fasta
F_notti
Total records: 67777
Converted records: 10831
Written: /kfish_clust/pep_renamed/Fnotti.pep.fasta
F_heteroclitusMDPL
Total records: 181552
Converted records: 31062
Written: /kfish_clust/pep_renamed/FheteroclitusMDPL.pep.fasta
A_xenica
Total records: 157368
Converted records: 30506
Written: /kfish_clust/pep_renamed/Axenica.pep.fasta
F_notatus
Total records: 175732
Converted records: 36945
Written: /kfish_clust/pep_renamed/Fnotatus.pep.fasta
F_catanatus
Total records: 160559
Converted records: 32605
Written: /kfish_clust/pep_renamed/Fcatanatus.pep.fasta
L_goodei
Total records: 148034
Converted records: 27305
Written: /kfish_clust/pep_renamed/Lgoodei.pep.fasta
F_grandis
Total records: 214281
Converted records: 36596
Written: /kfish_clust/pep_renamed/Fgrandis.pep.fas

# scrap code, do not use

In [18]:
fasta_file = basedir + "F_diaphanus.trinity_out.Trinity.fasta.transdecoder.pep.fasta"
new_filename = renamed + "Fdiaphanus.pep.fasta"
genusspecies="Fdiaphanus"
total_records = 0
converted_records = 0
repeated = 0
matchIDs = {}
for n, record in enumerate(screed.open(fasta_file)):
    if n==0:
        record_info = record.name.split("::")
        dammitID = record_info[1]
        dammit_num = int(dammit_ID.split("_")[1])
        type_info = record_info[5].split(" ")
        orf_type = type_info[3].split(":")[1]
        length = int(type_info[4].split(":")[1])
        TrinityID = conversions.loc[dammitID,'TrinityID']
        name = """{}-{}""".format(genusspecies,TrinityID)
        last_dammitID = dammitID
        last_TrinityID = TrinityID
        last_orf_type = orf_type
        last_length = int(length)
        last_type_info = type_info
        last_recordsequence = record.sequence
        last_name = name
    if n=>1 and n<20:
        record_info = record.name.split("::")
        dammitID = record_info[1]
        dammit_num = int(dammit_ID.split("_")[1])
        type_info = record_info[5].split(" ")
        orf_type = type_info[3].split(":")[1]
        length = int(type_info[4].split(":")[1])
        # assume transcripts that are the same are in order
        # check if complete or partial
        # keep complete over partial
        # keep longest
        # anything repeated 3 or more times? Yes
        TrinityID = conversions.loc[dammitID,'TrinityID']
        #print(TrinityID)
        if dammitID == last_dammitID:
            repeated +=1 
            #print("repeated",repeated)
            #print(dammitID)
            #print("New is duplicate from last record:",type_info,last_type_info)
            if orf_type == "complete" and last_orf_type == "complete":
                if length > last_length:
                    #print("keep new:",type_info, "not previous:",last_type_info)
                    name = """{}-{}""".format(genusspecies,TrinityID)
                else:
                    #print("keep previous:",last_type_info, "not new:",type_info)
                    name = """{}-{}""".format(genusspecies,last_TrinityID)
            elif orf_type == "complete" and last_orf_type.endswith("partial"):
                #print("keep new:",type_info, "not old:",last_type_info)
                name = """{}-{}""".format(genusspecies,TrinityID)
            elif orf_type.endswith("partial") and last_orf_type == "complete":
                #print("keep old:",last_type_info,"not new:",type_info)
                name = """{}-{}""".format(genusspecies,last_TrinityID)
            elif orf_type.endswith("partial") and last_orf_type.endswith("partial"):
                if length > last_length:
                    #print("keep new:",type_info, "not previous:",last_type_info)
                    name = """{}-{}""".format(genusspecies,TrinityID)
                else:
                    #print("keep previous:",last_type_info, "not new:",type_info)
                    name = """{}-{}""".format(genusspecies,last_TrinityID)
        else:
            name = """{}-{}""".format(genusspecies,TrinityID)
            #print(name)
            converted_records +=1
            repeated = 0
        last_name = name
        total_records +=1
        last_recordsequence = record.sequence
        last_dammitID = dammitID
        last_TrinityID = TrinityID
        last_orf_type = orf_type
        last_length = int(length)
        last_type_info = type_info
print(total_records)
print(converted_records)
        #fp.write(">{name}\n{seq}\n".format(name=name, seq=record.sequence))

SyntaxError: invalid syntax (<ipython-input-18-294e8051d563>, line 25)